In [2]:
#Importing necessary libraries (These are all the required libraries to run the full notebook)
import pandas as pd
import numpy as np 
import datetime
import ast
import plotly.express as px #only needed for plots
from sklearn.linear_model import Ridge

# Importing our custom functions, please make sure that the files are in the same directory as this notebook
from discounts import disc_per_day
from waste_functions import waste_analysis
#reading in the four datasets
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")
#----- Deals with products having sold in more than one size
tup_list2 = [('Blauwe bessen', '150g'), ('Rundergehakt', '300g '), ('Unox Gelderse rookworst', '285g'), ('Biologisch rundergehakt', '300g')]
for pair in tup_list2:
    transactions.loc[transactions[(transactions['description'] == pair[0]) & (transactions['size'] == pair[1])].index, 'description'] = pair[0] + '.1'
    products.loc[products[(products['description'] == pair[0]) & (products['size'] == pair[1])].index, 'description'] = pair[0] + '.1'


In [ ]:
discounts_per_day = disc_per_day(transactions) #returns a table with the nr of products on discount for each day 

## Start the waste optimization
First pick a product and run the following cells and at the end it will produce the table and plots we used in our presentation and report

In [ ]:
product = "Biologische brocolli" #product we want to analyze MODIFY to get results for another product

In [ ]:
full_output_table = waste_analysis(inventory = inventory, transactions = transactions, df_product=products, product=product, discount_per_day=discounts_per_day)


In [ ]:
full_output_table.head() #output table used for our main waste analysis and optimization

In [ ]:
# >>>> Cells only needed for the plots
# NOTE: The created plots are for one product chosen at the beginning of the notebook
results = pd.DataFrame({"Metric":["Loss revenue", "Loss profit", "Waste cost","Loss revenue", "Loss profit", "Waste cost"], "Method":["Actual", "Actual", "Actual", "Predicted w/discounts", "Predicted w/discounts", "Predicted w/discounts"], "values":output_full[money_cols].sum().values.round(1)})

In [ ]:
# >>>> Barchart for revenue, profit loss and waste costs with and without optimization
fig = px.bar(results, x="Metric", y='values', color="Method", barmode='group', text="values")
fig.update_layout(showlegend=False, xaxis_title="Product",
    yaxis_title="Out of Stock Event Count",width=1000,
    height=700, title={
                'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        paper_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
# >>>> Linechart for revenue loss per week
output_copy = output_full.groupby("week").sum()
output_copy["MA7"] = output_copy['expected loss revenue'].rolling(7).mean()
output_copy["MA7_exp"] = output_copy["loss revenue"].rolling(7).mean()
fig = px.line(output_copy, x=output_copy.index, y="MA7_exp", title="Loss of revenue for biological  vegetables per week")
fig.add_scatter(x=output_copy.index, y=output_copy["MA7"], mode="lines", name="With discounts applied")
fig.update_layout(showlegend=False, xaxis_title="Week nr",
    yaxis_title="Revenue loss in €",width=1000,
    height=700, title={
        
                'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        paper_bgcolor='rgba(0,0,0,0)')
fig.show()